In [4]:
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/Colab Notebooks/Weather classification Using Transfer Learning/dataset/data.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/Weather classification Using Transfer Learning/dataset/data.zip
   creating: train/
   creating: train/sunrise/
  inflating: train/sunrise/sunrise1.jpg  
  inflating: train/sunrise/sunrise10.jpg  
  inflating: train/sunrise/sunrise100.jpg  
  inflating: train/sunrise/sunrise101.jpg  
  inflating: train/sunrise/sunrise102.jpg  
  inflating: train/sunrise/sunrise103.jpg  
  inflating: train/sunrise/sunrise104.jpg  
  inflating: train/sunrise/sunrise105.jpg  
  inflating: train/sunrise/sunrise106.jpg  
  inflating: train/sunrise/sunrise107.jpg  
  inflating: train/sunrise/sunrise108.jpg  
  inflating: train/sunrise/sunrise109.jpg  
  inflating: train/sunrise/sunrise11.jpg  
  inflating: train/sunrise/sunrise110.jpg  
  inflating: train/sunrise/sunrise111.jpg  
  inflating: train/sunrise/sunrise112.jpg  
  inflating: train/sunrise/sunrise113.jpg  
  inflating: train/sunrise/sunrise114.jpg  
  inflating: train/sunrise/sunrise115.jpg  
  infla

In [3]:
imageSize = [180, 180]

trainPath = r"/content/train"

testPath = r"/content/test"


In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = [.99, 1.01],
                                   brightness_range=[0.8, 1.2],
                                   data_format="channels_last",
                                   fill_mode="constant",
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (180, 180),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (180,180),
                                            batch_size = 64,
                                            class_mode = 'categorical')

Found 1200 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [8]:
from tensorflow.keras.applications.vgg19 import VGG19
VGG19 = VGG19(input_shape=imageSize + [3], include_top=False, weights="imagenet")

80134624/80134624 [==============================] - 1s 0us/step


In [9]:
for layer in VGG19.layers:
    layer.trainable=False

In [10]:
x = Flatten()(VGG19.output)

In [11]:
prediction = Dense(5, activation='softmax')(x)

In [12]:
model = Model(inputs=VGG19.input, outputs=prediction)

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [17]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-17-070b10e8c207>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
19/19 [==============================] - 38s 1s/step - loss: 1.2036 - accuracy: 0.5458 - val_loss: 0.8161 - val_accuracy: 0.7233
Epoch 2/50
19/19 [==============================] - 22s 1s/step - loss: 0.6130 - accuracy: 0.7758 - val_loss: 0.5963 - val_accuracy: 0.8100
Epoch 3/50
19/19 [==============================] - 21s 1s/step - loss: 0.4537 - accuracy: 0.8675 - val_loss: 0.6144 - val_accuracy: 0.7967
Epoch 4/50
19/19 [==============================] - 22s 1s/step - loss: 0.3786 - accuracy: 0.8925 - val_loss: 0.5267 - val_accuracy: 0.8333
Epoch 5/50
19/19 [==============================] - 22s 1s/step - loss: 0.3249 - accuracy: 0.9117 - val_loss: 0.5551 - val_accuracy: 0.8300
Epoch 6/50
19/19 [==============================] - 22s 1s/step - loss: 0.2723 - accuracy: 0.9242 - val_loss: 0.4016 - val_accuracy: 0.8700
Epoch 7/50
19/19 [==============================] - 22s 1s/step - loss: 0.2445 - accuracy: 0.9325 - val_loss: 0.3708 - val_accuracy: 0.8767
Epoch 8/50
19/19 [==

In [18]:
model.save('wcv.h5')

In [19]:
loss, accuracy = model.evaluate(test_set,
                                    steps=11, 
                                    verbose=2, 
                                    use_multiprocessing=True, 
                                    workers=2)
print(f'Model performance on test images:\nAccuracy = {accuracy}\nLoss = {loss}')

11/11 - 5s - loss: 0.3115 - accuracy: 0.9111 - 5s/epoch - 497ms/step
Model performance on test images:
Accuracy = 0.9111445546150208
Loss = 0.3114546537399292


**Testing the Model**


In [20]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [21]:
model = load_model('/content/wcv.h5')

In [22]:
img=image.load_img(r'/content/test/cloudy/cloudy290.jpg',target_size=(180,180))
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
img_data.shape
model.predict(img_data)
output=np.argmax(model.predict(img_data), axis=1)
output

1/1 [==============================] - 0s 30ms/step


array([0])

In [23]:
img=image.load_img(r'/content/test/foggy/foggy296.jpg',target_size=(180,180))
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
img_data.shape
model.predict(img_data)
output=np.argmax(model.predict(img_data), axis=1)
output

1/1 [==============================] - 0s 22ms/step


array([1])

In [24]:
img=image.load_img(r'/content/test/rainy/rain273.jpg',target_size=(180,180))
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
img_data.shape
model.predict(img_data)
output=np.argmax(model.predict(img_data), axis=1)
output

1/1 [==============================] - 0s 22ms/step


array([2])

In [25]:
img=image.load_img(r'/content/test/shine/shine229.jpg',target_size=(180,180))
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
img_data.shape
model.predict(img_data)
output=np.argmax(model.predict(img_data), axis=1)
output

1/1 [==============================] - 0s 20ms/step


array([3])

In [26]:
img=image.load_img(r'/content/test/sunrise/sunrise307.jpg',target_size=(180,180))
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
img_data.shape
model.predict(img_data)
output=np.argmax(model.predict(img_data), axis=1)
output

1/1 [==============================] - 0s 20ms/step


array([4])

In [27]:
img=image.load_img(r'/content/test/sunrise/sunrise297.jpg',target_size=(180,180))
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
img_data.shape
model.predict(img_data)
output=np.argmax(model.predict(img_data), axis=1)
output

1/1 [==============================] - 0s 21ms/step


array([4])

In [28]:
img=image.load_img(r'/content/test/rainy/rain250.jpg',target_size=(180,180))
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape
img_data.shape
model.predict(img_data)
output=np.argmax(model.predict(img_data), axis=1)
output

1/1 [==============================] - 0s 22ms/step


array([2])